In [2]:
import os
from datetime import datetime
from tracktool import load_graph
import networkx as nx
import igraph
import numpy as np
import pandas as pd

In [3]:
DATA_ROOT = '/home/draga/PhD/data/cell_tracking_challenge/ST_Segmentations/'
DS_NAME = 'Fluo-N2DL-HeLa/'
SEQ = '01_ST'
GT_PATH = os.path.join(DATA_ROOT, DS_NAME, '01_GT/TRA/')
SEG_PATH = os.path.join(DATA_ROOT, DS_NAME, SEQ, 'SEG/')

In [5]:
seg_ims, flow_graph = load_graph(SEG_PATH)
flow_graph.solve()

Making migration & division edges: 100%|██████████| 91/91 [00:05<00:00, 15.74it/s]


Set parameter Username
Academic license - for non-commercial use only - expires 2024-06-18
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (linux64)

CPU model: Intel(R) Core(TM) i7-10510U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 25815 rows, 110358 columns and 424220 nonzeros
Model fingerprint: 0x69dd8f84
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-02, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 139 rows and 2 columns
Presolve time: 0.24s
Presolved: 25676 rows, 110356 columns, 422722 nonzeros

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Ordering time: 0.03s

Barrier statistics:
 AA' NZ     : 3.039e+05
 Factor NZ  : 1.262e+06 (roughly 70 MB of memory)
 Factor Ops : 1.678e+08 (less than 1 second per iteration)
 Threads    : 3

Barrier performed 0 iterations in 0.42 

In [ ]:
# load into napari